# KNN

In [1]:
# on importe les differentes librairies
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer


#pour les plots en fonction des distances euclidienne/cosine
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity

# on importe les différentes librairies surprise de scikit
from surprise import SVD
from surprise import dataset
from surprise import Reader
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms.knns import KNNBasic, KNNWithMeans

# on importe notre fonction de tri
from tri_threshold import filter_reviews

# Téléchargement des Stop Words Français depuis NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

french_stop_words = stopwords.words('french')

# on importe notre bdd
avis = pd.read_csv("BDD/avis.csv", index_col="Unnamed: 0")
avis_norm = pd.read_csv("BDD/avis_norm.csv")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lazra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
avis.head()

,author,date_published,title_review,note,title,url,comment
0,Monsieur Guillaume,2021-01,Voyages sur les ailes des papillons,8,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,"Lorsque le jeu est jeu, bon, réflexif, joli po..."
1,morlockbob,2020-10,le festival de Michoacan,7,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,Comment continuer après un mega hit ? Simpleme...
2,SwatSh,2021-02,Vivez la migration des monarques,7,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,"Vin d'jeu: Avec Mariposas, Elizabeth Hargrave ..."
3,Timi JeuxATheme,2020-11,Bon,8,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,NaN
4,prunelles,2021-05,Envolez-moi,9,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,"Très joli bijou que ce jeu-là ! Le matériel, l..."


In [3]:
avis_norm.head()

,Unnamed: 0.1,Unnamed: 0,author,date_published,title_review,note,title,url,comment
0,0,0,Monsieur Guillaume,2021-01,Voyages sur les ailes des papillons,0.8,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,"Lorsque le jeu est jeu, bon, réflexif, joli po..."
1,1,1,morlockbob,2020-10,le festival de Michoacan,0.7,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,Comment continuer après un mega hit ? Simpleme...
2,2,2,SwatSh,2021-02,Vivez la migration des monarques,0.7,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,"Vin d'jeu: Avec Mariposas, Elizabeth Hargrave ..."
3,3,3,Timi JeuxATheme,2020-11,Bon,0.8,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,NaN
4,4,4,prunelles,2021-05,Envolez-moi,0.9,Mariposas,https://www.trictrac.net/jeu-de-societe/maripo...,"Très joli bijou que ce jeu-là ! Le matériel, l..."


**SPARSITE**

In [4]:
# créer des mappings
user_ids = avis['author'].unique().tolist()
user_id_map = {user_id: index for index, user_id in enumerate(user_ids)}
item_ids = avis['title'].unique().tolist()
item_id_map = {item_id: index for index, item_id in enumerate(item_ids)}

# Définissez num_users et num_items ici
num_users = len(user_ids)
num_items = len(item_ids)

row_indices = avis['author'].map(user_id_map)
col_indices = avis['title'].map(item_id_map)
values = avis['note']


values = values.astype(float)

# Création de la matrice sparse
matrice_sparse = csr_matrix((values, (row_indices, col_indices)), shape=(num_users, num_items))

print(matrice_sparse)

  (0, 0)	8.0
  (0, 3)	8.0
  (0, 8)	6.0
  (0, 9)	8.0
  (0, 13)	7.0
  (0, 40)	9.0
  (0, 55)	8.0
  (0, 60)	8.0
  (0, 64)	9.0
  (0, 70)	9.0
  (0, 71)	9.0
  (0, 89)	8.0
  (0, 93)	8.0
  (0, 94)	8.0
  (0, 105)	7.0
  (0, 108)	8.0
  (0, 110)	9.0
  (0, 161)	8.0
  (0, 190)	7.0
  (0, 201)	6.0
  (0, 206)	7.0
  (0, 209)	8.0
  (0, 210)	8.0
  (0, 543)	8.0
  (0, 582)	8.0
  :	:
  (2561, 3004)	6.0
  (2562, 1902)	4.0
  (2562, 1924)	6.0
  (2562, 2225)	6.0
  (2562, 2324)	10.0
  (2562, 2372)	6.0
  (2562, 2475)	10.0
  (2562, 2601)	4.0
  (2562, 2642)	8.0
  (2562, 2853)	8.0
  (2562, 3008)	8.0
  (2563, 2032)	8.0
  (2563, 2102)	10.0
  (2563, 2148)	10.0
  (2563, 2205)	8.0
  (2563, 2359)	8.0
  (2563, 2375)	6.0
  (2563, 2386)	8.0
  (2563, 2411)	6.0
  (2563, 2442)	8.0
  (2563, 2464)	8.0
  (2563, 2549)	8.0
  (2563, 2741)	10.0
  (2563, 2890)	2.0
  (2563, 2908)	6.0


## KNN Hashage Sensible au Locality (LSH) 

**Extraction des caractéristiques** 
1) Convertir les commentaires en vecteurs numériques.Term Frequency-Inverse Document Frequency  TF-IDF (TF-IDF)\
2) Intégrée la  note comme une dimension supplémentaire du vecteur 

In [5]:
# Remplacer les valeurs NaN par une chaîne vide
avis_norm['comment'] = avis_norm['comment'].fillna('')

# Utiliser les stop words français obtenus de NLTK
tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words=french_stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(avis_norm['comment']).toarray()
normalized_notes = avis_norm['note'].values.reshape(-1, 1) / 10

# Intégration des notes comme une dimension supplémentaire des vecteurs TF-IDF
user_vectors = np.hstack((tfidf_matrix, normalized_notes))


L'idée derrière TF-IDF est de refléter l'importance des mots en tenant compte non seulement de leur fréquence dans un document spécifique (TF) mais aussi de leur fréquence dans l'ensemble du corpus (IDF).
Ici on utilise (NLP)
\
TF : La fréquence d'un mot dans un document
IDF : Une mesure de l'importance du mot dans l'ensemble du corpus.
\
TF-IDF=TF(t,d)×IDF(t,D)
où t est le terme (mot), d est un document, et D est l'ensemble des documents.

Utilisation de Annoy (Approximate Nearest Neighbors Oh Yeah)

In [6]:
from annoy import AnnoyIndex

# La dimension de chaque vecteur est le nombre de caractéristiques TF-IDF plus la dimension pour la note
f = user_vectors.shape[1]

# Initialiser Annoy avec le nombre de dimensions f et en utilisant la distance euclidienne
t = AnnoyIndex(f, 'euclidean')

# Ajouter tous les vecteurs utilisateur à l'index
for i in range(len(user_vectors)):
    t.add_item(i, user_vectors[i])

# Construire l'index avec 10 arbres. Plus il y a d'arbres, plus la requête est précise mais plus lente.
t.build(10)

# Sauvegarder l'index sur le disque
t.save('user_vectors_index.ann')

# Charger l'index, si nécessaire, avec
u = AnnoyIndex(f, 'euclidean')
u.load('user_vectors_index.ann')

# Trouver les 10 voisins les plus proches de l'utilisateur 0
neighbors = u.get_nns_by_item(0, 10)  # Il renvoie les indices des 10 voisins les plus proches

# Afficher les indices des voisins
print("Indices des voisins les plus proches :", neighbors)


Indices des voisins les plus proches : [0, 28812, 33916, 35729, 122357, 6200, 29887, 89159, 71779, 110902]


La distance angulaire est une mesure qui est étroitement liée à la similarité cosinus, particulièrement adaptée pour comparer la direction (mais pas la magnitude) des vecteurs.

In [7]:
# La dimension de chaque vecteur est le nombre de caractéristiques TF-IDF plus la dimension pour la note
f = user_vectors.shape[1]

# Initialiser Annoy avec le nombre de dimensions f et en utilisant la distance angulaire (cosinus)
t = AnnoyIndex(f, 'angular')

# Ajouter tous les vecteurs utilisateur à l'index
for i in range(len(user_vectors)):
    t.add_item(i, user_vectors[i])

# Construire l'index avec 10 arbres. Plus il y a d'arbres, plus la requête est précise mais plus lente.
t.build(10)

# Sauvegarder l'index sur le disque
t.save('user_vectors_index_cosine.ann')

# Charger l'index, si nécessaire, avec
u = AnnoyIndex(f, 'angular')
u.load('user_vectors_index_cosine.ann')

# Trouver les 10 voisins les plus proches de l'utilisateur 0 en utilisant la distance cosinus
neighbors_cosine = u.get_nns_by_item(0, 10)  # Il renvoie les indices des 10 voisins les plus proches

# Afficher les indices des voisins
print("Indices des voisins les plus proches (cosinus) :", neighbors_cosine)


Indices des voisins les plus proches (cosinus) : [0, 89159, 27576, 53717, 66396, 68538, 74869, 81874, 82809, 123563]


In [11]:
print("par distance euclidienne\n")

for neighbor in neighbors:
    print("Utilisateur :", avis_norm.iloc[neighbor]['author'])
    print("Jeu :", avis_norm.iloc[neighbor]['title'])
    print("Commentaire :", avis_norm.iloc[neighbor]['comment'])
    print("Note :", avis_norm.iloc[neighbor]['note'])
    print("---")
    


par distance euclidienne

Utilisateur : Monsieur Guillaume
Jeu : Mariposas
Commentaire : Lorsque le jeu est jeu, bon, réflexif, joli pour qui est sensible à ce style d'illustration... Lorsque l'on a envie d'y rejouer encore pour essayer autrement, et revivre cette histoire de papillons et de voyages... Et lorsqu'en plus, au détour des règles, on s'intéresse également au propos et à la sensibilisation, légère comme les ailes du dit-papillons et pourtant aussi bien amené, alors effectivement, il y a là un très bon jeu, ludiquement parlant (et il est possible de s'arrêter là) mais également très bon pour l'ouverture des yeux des joueuses et joueurs au monde qui nous entoure... Ça, c'est deux fois bon !
Note : 0.8
---
Utilisateur : Hakuin
Jeu : Rush Hour
Commentaire : un très bon jeu labyrinthique.
Note : 0.8
---
Utilisateur : hrvlef
Jeu : Wizard
Commentaire : Très bon jeu de plis et de contrat avec un brin d'imprévisible. Le whist revu et corrigé.
Note : 0.8
---
Utilisateur : Serval-Wolve

In [12]:
print("par distance cosine\n")
for neighbor in neighbors_cosine:
    print("Utilisateur :", avis_norm.iloc[neighbor]['author'])
    print("Jeu :", avis_norm.iloc[neighbor]['title'])
    print("Commentaire :", avis_norm.iloc[neighbor]['comment'])
    print("Note :", avis_norm.iloc[neighbor]['note'])
    print("---")

par distance cosine

Utilisateur : Monsieur Guillaume
Jeu : Mariposas
Commentaire : Lorsque le jeu est jeu, bon, réflexif, joli pour qui est sensible à ce style d'illustration... Lorsque l'on a envie d'y rejouer encore pour essayer autrement, et revivre cette histoire de papillons et de voyages... Et lorsqu'en plus, au détour des règles, on s'intéresse également au propos et à la sensibilisation, légère comme les ailes du dit-papillons et pourtant aussi bien amené, alors effectivement, il y a là un très bon jeu, ludiquement parlant (et il est possible de s'arrêter là) mais également très bon pour l'ouverture des yeux des joueuses et joueurs au monde qui nous entoure... Ça, c'est deux fois bon !
Note : 0.8
---
Utilisateur : Biff
Jeu : Caylus Magna Carta
Commentaire : du bon, même très bon. Un futur incontournable en 1h.
Note : 1.0
---
Utilisateur : M.PinK
Jeu : Palast Geflüster
Commentaire : ...que bon!\nVoila
Note : 0.8
---
Utilisateur : kumagron
Jeu : Civilisation
Commentaire : Looooo